In [2]:
import pandas as pd
import jpholiday

pd.set_option("display.max_columns",None)

In [3]:
df = pd.read_csv(r"..\data\df_preprocessed.csv")
df.head(3)

,Year,Date,Audience,Result,Pitcher,GameTime,Average_Temperature (℃),Total_Precipitation (mm),Average_wind_speed(m/s),Home_Score,Away_Score,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday,Opponent_DeNA,Opponent_オリックス,Opponent_ヤクルト,Opponent_ロッテ,Opponent_中日,Opponent_巨人,Opponent_広島,Opponent_日本ハム,Opponent_楽天,Opponent_西武,Opponent_阪神
0,2015,2015-03-27,38500,0,攝津,197,12.2,0.0,1.7,1,3,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,2015,2015-03-28,37397,1,スタンリッジ,157,16.1,0.0,2.2,4,2,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,2015,2015-03-29,38118,0,中田,201,16.1,0.0,2.5,4,5,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [4]:
def feature_engineering(df):
    # Date列をdatetime型に変換
    df['Date'] = pd.to_datetime(df['Date'])
    
    # 祝日フラグの作成
    df['Is_Holiday'] = df['Date'].apply(lambda x: 1 if jpholiday.is_holiday(x) else 0)
    
    # 試合数の作成（同年内で何試合目か）
    df['Match_Number'] = df.groupby('Year').cumcount() + 1
    
    # 雨量0のフラグ
    df['Rain_Zero_Flag'] = df['Total_Precipitation (mm)'].apply(lambda x: 1 if x == 0 else 0)

    # ドームの定員
    df['Capacity'] = df['Year'].apply(lambda x: 38530 if x < 2019 else 40178)

    # 占有率
    df["Occupancy"] = df["Audience"] / df['Capacity']
    
    return df
df_featured = feature_engineering(df)
df_featured.head(3)

,Year,Date,Audience,Result,Pitcher,GameTime,Average_Temperature (℃),Total_Precipitation (mm),Average_wind_speed(m/s),Home_Score,Away_Score,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday,Opponent_DeNA,Opponent_オリックス,Opponent_ヤクルト,Opponent_ロッテ,Opponent_中日,Opponent_巨人,Opponent_広島,Opponent_日本ハム,Opponent_楽天,Opponent_西武,Opponent_阪神,Is_Holiday,Match_Number,Rain_Zero_Flag,capacity,occupancy
0,2015,2015-03-27,38500,0,攝津,197,12.2,0.0,1.7,1,3,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,38530,0.999221
1,2015,2015-03-28,37397,1,スタンリッジ,157,16.1,0.0,2.2,4,2,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,1,38530,0.970594
2,2015,2015-03-29,38118,0,中田,201,16.1,0.0,2.5,4,5,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3,1,38530,0.989307


In [5]:
df_featured.to_csv(r"..\data\df_featured.csv",index=False)